# Datos climatológicos & hidrológicos

In [1]:
#Paquetes

import pandas as pd
import numpy as np
import datetime 

#### Función de distancias mínimas

In [2]:
#Función distancias mínimas

from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lat2, lon1, lon2): 
      
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

def dist_min(df1,column_lat1,column_lat2,column_lng1,column_lng2,index,name):
    
    #Add column containing distances 
    df1[name]=df1.apply(lambda x: distance(x[column_lat1], x[column_lat2], x[column_lng1], x[column_lng2]), axis=1)
    #Selecting the min distance per id
    df1_min = df1.groupby([index]).agg({name: 'min'})
    df2 = pd.merge(df1, df1_min, on = index, how ='inner')
    df = df2[(df2[name+'_x']==df2[name+'_y']) | df2[name+'_x'].isna()]
    return(df)

#### Función cambio de minutos y segundos a grados con decimales

In [3]:
import re

def dms2dec(dms_str):
    
    dms_str = re.sub(r'\s', '', dms_str)
        
    numbers = [*filter(len, re.split('\D+', dms_str, maxsplit=4))]

    degree = numbers[0]
    minute = numbers[1] if len(numbers) >= 2 else '0'
    second = numbers[2] if len(numbers) >= 3 else '0'
    frac_seconds = numbers[3] if len(numbers) >= 4 else '0'
    
    second += "." + frac_seconds
    sign = -1 if int(degree)<0 else 1
    return sign * (int(degree) + float(minute) / 60 + float(second) / 3600)

## Carga de datos

### Datos incendios

In [4]:
#Carga de datos
df = pd.read_csv('data.csv',index_col=[0])
df['coord'] = list(zip(df.lat, df.lng))
df['mes']=df['fecha'].str[:7]
df['num_mes']=df['fecha'].str[5:7]
df['ano']=df['fecha'].str[:4]

### Datos climatológicos

In [5]:
#Carga de datos
tiempo=pd.read_excel('tiempo.xlsx')
tiempo.columns = map(lambda x: str(x).lower(), tiempo.columns) 
tiempo['provincia'] = tiempo['provincia'].str.upper() 
tiempo['estación'] = tiempo['estación'].str.upper() 

#inclusión del código de la provincia:
tiempo['idprovincia'] = tiempo['provincia'].apply(lambda x: 15 if x=='A CORUÑA' else (32 if x=='OURENSE' else (27 if x=='LUGO' else 36)))
#inclusión del mes-año
tiempo['fecha']=tiempo['fecha'].astype(str)
tiempo['mes']=tiempo['fecha'].str[:7]
tiempo

,fecha,provincia,estación,latitud,longitud,temp_media,temp_max_med,temp_min_med,prec_acu,hum_med,hum_max,hum_min,v_viento_med,presion,lluvia,helada,idprovincia,mes
0,2001-02-01,A CORUÑA,CIS FERROL,43.491467,-8.252274,10.8,14.7,7.5,97.0,78.0,92.0,58.0,3.0,1015.0,13.0,0.0,15,2001-02
1,2001-02-01,A CORUÑA,CORRUBEDO,42.555164,-9.028601,11.8,15.2,9.1,98.0,76.0,92.0,59.0,5.0,1017.0,10.0,0.0,15,2001-02
2,2001-02-01,A CORUÑA,MABEGONDO,43.241367,-8.262225,9.0,15.4,3.4,95.0,83.0,99.0,54.0,2.0,1010.0,15.0,7.0,15,2001-02
3,2001-02-01,A CORUÑA,MARCO DA CURRA,43.343082,-7.894400,6.2,10.6,2.9,NaN,84.0,98.0,60.0,6.0,NaN,NaN,3.0,15,2001-02
4,2001-02-01,A CORUÑA,MURALLA,42.745583,-8.776306,8.3,11.3,5.4,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,15,2001-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14795,2015-12-01,PONTEVEDRA,TORREQUINTÁNS,42.537130,-8.716114,13.2,17.1,9.7,151.0,73.0,85.0,55.0,8.0,1017.0,24.0,0.0,36,2015-12
14796,2015-12-01,PONTEVEDRA,TREMOEDO,42.539917,-8.784433,13.1,16.8,9.7,141.0,75.0,88.0,57.0,5.0,1017.0,18.0,0.0,36,2015-12
14797,2015-12-01,PONTEVEDRA,VIGO,42.225945,-8.713502,13.8,17.5,10.9,109.0,84.0,98.0,61.0,NaN,NaN,16.0,0.0,36,2015-12
14798,2015-12-01,PONTEVEDRA,VIGO-CAMPUS,42.169926,-8.686054,11.6,13.9,9.7,154.0,75.0,90.0,59.0,20.0,971.0,18.0,0.0,36,2015-12


### Datos hidrológicos

In [6]:
#Carga de embalses
#(Lugo, A Coruña y Ourense)
embalses = pd.read_excel('embalses.xlsx')
#(Pontevedra)
emb_pontevedra= pd.read_csv('embalses_pontevedra.csv')

#Adaptación de los embalses de Lugo, A Coruña y Ourense a los de Pontevedra
embalses.columns = map(lambda x: str(x).lower(), embalses.columns)
embalses['nombre'] = embalses['nombre'].str.upper()
embalses['provincia'] = embalses['provincia'].str.upper()
embalses['río'] = embalses['río'].str.upper()
embalses = embalses.rename(columns={'capacidad (hm³)' : 'capacidad_emb', 'superficie (ha)':'sup_emb'})
embalses['lat_embalse'] = embalses['latitud'].apply(dms2dec)
embalses['lng_embalse'] = - embalses['longitud'].apply(dms2dec)
embalses['idprovincia'] = embalses['provincia'].apply(lambda x: 15 if x=='A CORUÑA' else (32 if x=='OURENSE' else (27 if x=='LUGO' else 36)))
embalses = embalses[['idprovincia','nombre','río','capacidad_emb', 'sup_emb', 'lat_embalse','lng_embalse']]

#Unión de los embalses de Galicia
embalses = pd.concat([embalses,emb_pontevedra], ignore_index=True)
embalses

,idprovincia,nombre,río,capacidad_emb,sup_emb,lat_embalse,lng_embalse
0,15,ENCORO DO EUME,EUME,123.00,425.00,43.400000,-8.000000
1,15,ENCORO DA FERVENZA,XALAS,104.00,1250.00,43.000000,-8.950000
2,15,ENCORO DE PONTEMOUROS,ULLA,297.00,1204.98,42.850000,-8.150000
3,27,ENCORO DE BELESAR,MIÑO,645.56,1828.00,42.683333,-7.700000
4,27,ENCORO DOS PEARES,MIÑO,182.00,600.00,42.466667,-7.716667
5,32,ENCORO DE ALBARELLOS,AVIA,90.73,314.02,42.383333,-8.233333
6,32,ENCORO DAS CONCHAS,LIMIA,80.00,645.00,41.933333,-8.016667
7,32,ENCORO DAS PORTAS,CAMBA,535.71,1183.00,42.083333,-7.233333
8,32,ENCORO DE PRADA,JARES,122.00,605.24,42.300000,-7.033333
9,32,ENCORO DE SALAS,SALAS,86.67,685.50,41.916667,-7.916667


In [7]:
#Carga de rios
rios=pd.read_excel('RIOS_GALICIA.xlsx')
rios.columns = map(lambda x: str(x).lower(), rios.columns)
rios['rio'] = rios['rio'].str.upper()
rios['codigo'] = rios['codigo'].apply(np.int64)

#Conteo de ríos por municipio
num_rios =rios.groupby('codigo')['rio'].agg(['count'])
rios=  pd.merge(rios, num_rios, on = ['codigo'], how ='left')
rios = rios[['codigo','count']].drop_duplicates()
rios

,codigo,count
0,15041,1
1,15024,1
2,15019,1
3,15029,1
4,15068,1
...,...,...
214,32044,1
216,32063,1
219,27024,2
226,27045,1


#### Unión de incendios con embalses

In [8]:
#Unión de Data con embalses por provincias
df_emb = pd.merge(df, embalses, on = ['idprovincia'], how ='inner')

In [9]:
df=dist_min(df_emb,'lat','lat_embalse','lng','lng_embalse','id','dist_emb')

#### Unión de incendios+embalses con ríos

In [10]:
#Unión de Data con embalses por provincias y con ríos por municipio 
df = pd.merge(df, rios, on = 'codigo', how ='left')
#Cambio de nombre de columna
df = df.rename(columns={'count' : 'num_rios'})
df['num_rios'] = df['num_rios'].fillna(0)

#### Unión de incendios+embalses+ríos con clima

In [11]:
#Unión de Data con embalses por provincias y con ríos por municipio y 
#con tiempo por provincias y mes
df =pd.merge(df, tiempo, on =['mes','idprovincia'], how ='inner')

In [12]:
df

,id,superficie,fecha_x,lat,lng,idprovincia,provincia_x,idmunicipio,municipio,causa,...,temp_max_med,temp_min_med,prec_acu,hum_med,hum_max,hum_min,v_viento_med,presion,lluvia,helada
0,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,14.7,7.5,97.0,78.0,92.0,58.0,3.0,1015.0,13.0,0.0
1,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,15.2,9.1,98.0,76.0,92.0,59.0,5.0,1017.0,10.0,0.0
2,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,15.4,3.4,95.0,83.0,99.0,54.0,2.0,1010.0,15.0,7.0
3,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,10.6,2.9,NaN,84.0,98.0,60.0,6.0,NaN,NaN,3.0
4,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,11.3,5.4,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357822,2015360819,2.26,2015-12-27 00:00:00-05:00,42.658303,-8.268949,36,PONTEVEDRA,52,SILLEDA,4,...,17.1,9.7,151.0,73.0,85.0,55.0,8.0,1017.0,24.0,0.0
357823,2015360819,2.26,2015-12-27 00:00:00-05:00,42.658303,-8.268949,36,PONTEVEDRA,52,SILLEDA,4,...,16.8,9.7,141.0,75.0,88.0,57.0,5.0,1017.0,18.0,0.0
357824,2015360819,2.26,2015-12-27 00:00:00-05:00,42.658303,-8.268949,36,PONTEVEDRA,52,SILLEDA,4,...,17.5,10.9,109.0,84.0,98.0,61.0,NaN,NaN,16.0,0.0
357825,2015360819,2.26,2015-12-27 00:00:00-05:00,42.658303,-8.268949,36,PONTEVEDRA,52,SILLEDA,4,...,13.9,9.7,154.0,75.0,90.0,59.0,20.0,971.0,18.0,0.0


In [13]:
df_final = dist_min(df, 'lat', 'latitud', 'lng', 'longitud', 'id','dist_est')
df_final

,id,superficie,fecha_x,lat,lng,idprovincia,provincia_x,idmunicipio,municipio,causa,...,prec_acu,hum_med,hum_max,hum_min,v_viento_med,presion,lluvia,helada,dist_est_x,dist_est_y
0,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,...,97.0,78.0,92.0,58.0,3.0,1015.0,13.0,0.0,29.186920,29.186920
11,2001150088,1.50,2001-02-24 00:00:00-05:00,42.936918,-9.114350,15,A CORUÑA,34,DUMBRÍA,4,...,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,34.817885,34.817885
15,2001150090,3.00,2001-02-25 00:00:00-05:00,42.643031,-8.939252,15,A CORUÑA,67,"POBRA DO CARAMIÑAL, A",4,...,98.0,76.0,92.0,59.0,5.0,1017.0,10.0,0.0,12.204302,12.204302
26,2001150094,1.50,2001-02-25 00:00:00-05:00,43.186836,-8.685470,15,A CORUÑA,19,CARBALLO,4,...,12.0,92.0,99.0,74.0,NaN,NaN,17.0,8.0,3.411028,3.411028
32,2001150111,3.80,2001-02-25 00:00:00-05:00,42.917476,-9.082862,15,A CORUÑA,45,MAZARICOS,4,...,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,31.468092,31.468092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357650,2015360747,1.45,2015-09-06 00:00:00-04:00,42.103959,-8.522512,36,PONTEVEDRA,50,SALVATERRA DE MIÑO,4,...,108.0,76.0,96.0,48.0,3.0,NaN,10.0,0.0,5.065956,5.065956
357693,2015360751,1.00,2015-09-06 00:00:00-04:00,42.157695,-8.397080,36,PONTEVEDRA,34,"NEVES, AS",4,...,90.0,68.0,88.0,40.0,2.0,999.0,10.0,0.0,5.750805,5.750805
357743,2015360770,5.94,2015-09-08 00:00:00-04:00,42.554722,-8.018586,36,PONTEVEDRA,16,DOZÓN,4,...,117.0,76.0,90.0,55.0,19.0,905.0,9.0,0.0,7.535815,7.535815
357778,2015360794,3.20,2015-09-28 00:00:00-04:00,42.243441,-8.322375,36,PONTEVEDRA,13,COVELO,4,...,137.0,73.0,89.0,52.0,4.0,975.0,6.0,0.0,9.613333,9.613333


In [14]:
pd.options.display.max_columns=None
df_final

,id,superficie,fecha_x,lat,lng,idprovincia,provincia_x,idmunicipio,municipio,causa,causa_desc,time_ctrl,time_ext,personal,medios,gastos,perdidas,codigo,superficie_total_concello,superficie_forestal_total,coord,mes,num_mes,ano,nombre,río,capacidad_emb,sup_emb,lat_embalse,lng_embalse,dist_emb_x,dist_emb_y,num_rios,fecha_y,provincia_y,estación,latitud,longitud,temp_media,temp_max_med,temp_min_med,prec_acu,hum_med,hum_max,hum_min,v_viento_med,presion,lluvia,helada,dist_est_x,dist_est_y
0,2001150021,5.00,2001-02-20 00:00:00-05:00,43.703581,-8.038777,15,A CORUÑA,22,CEDEIRA,2,Descuidos humanos,235,270,14,2,NaN,7013.0,15022,NaN,NaN,"(43.703581, -8.038777)",2001-02,02,2001,ENCORO DO EUME,EUME,123.0,425.00,43.400000,-8.000000,33.901004,33.901004,0.0,2001-02-01,A CORUÑA,CIS FERROL,43.491467,-8.252274,10.8,14.7,7.5,97.0,78.0,92.0,58.0,3.0,1015.0,13.0,0.0,29.186920,29.186920
11,2001150088,1.50,2001-02-24 00:00:00-05:00,42.936918,-9.114350,15,A CORUÑA,34,DUMBRÍA,4,Intencionado,470,530,14,1,NaN,1497.0,15034,NaN,NaN,"(42.936918, -9.11435)",2001-02,02,2001,ENCORO DA FERVENZA,XALAS,104.0,1250.00,43.000000,-8.950000,15.100302,15.100302,1.0,2001-02-01,A CORUÑA,MURALLA,42.745583,-8.776306,8.3,11.3,5.4,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,34.817885,34.817885
15,2001150090,3.00,2001-02-25 00:00:00-05:00,42.643031,-8.939252,15,A CORUÑA,67,"POBRA DO CARAMIÑAL, A",4,Intencionado,185,220,14,3,NaN,1882.0,15067,NaN,NaN,"(42.643031, -8.939252)",2001-02,02,2001,ENCORO DA FERVENZA,XALAS,104.0,1250.00,43.000000,-8.950000,39.702820,39.702820,0.0,2001-02-01,A CORUÑA,CORRUBEDO,42.555164,-9.028601,11.8,15.2,9.1,98.0,76.0,92.0,59.0,5.0,1017.0,10.0,0.0,12.204302,12.204302
26,2001150094,1.50,2001-02-25 00:00:00-05:00,43.186836,-8.685470,15,A CORUÑA,19,CARBALLO,4,Intencionado,125,135,5,0,NaN,1028.0,15019,NaN,NaN,"(43.186836, -8.68547)",2001-02,02,2001,ENCORO DA FERVENZA,XALAS,104.0,1250.00,43.000000,-8.950000,29.882779,29.882779,1.0,2001-02-01,A CORUÑA,RUS,43.156160,-8.685357,8.8,15.3,3.6,12.0,92.0,99.0,74.0,NaN,NaN,17.0,8.0,3.411028,3.411028
32,2001150111,3.80,2001-02-25 00:00:00-05:00,42.917476,-9.082862,15,A CORUÑA,45,MAZARICOS,4,Intencionado,1050,1051,14,1,NaN,3119.0,15045,NaN,NaN,"(42.917476, -9.082862)",2001-02,02,2001,ENCORO DA FERVENZA,XALAS,104.0,1250.00,43.000000,-8.950000,14.181046,14.181046,1.0,2001-02-01,A CORUÑA,MURALLA,42.745583,-8.776306,8.3,11.3,5.4,69.0,79.0,91.0,63.0,NaN,942.0,9.0,0.0,31.468092,31.468092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357650,2015360747,1.45,2015-09-06 00:00:00-04:00,42.103959,-8.522512,36,PONTEVEDRA,50,SALVATERRA DE MIÑO,4,Intencionado,92,486,16,3,2012.0,6198.0,36050,6254.0,3481.0,"(42.10395894, -8.52251246)",2015-09,09,2015,EIRAS,OITAVEN,22.0,149.00,42.346138,-8.490387,27.058684,27.058684,1.0,2015-09-01,PONTEVEDRA,ENTENZA,42.069332,-8.562408,17.7,25.3,12.0,108.0,76.0,96.0,48.0,3.0,NaN,10.0,0.0,5.065956,5.065956
357693,2015360751,1.00,2015-09-06 00:00:00-04:00,42.157695,-8.397080,36,PONTEVEDRA,34,"NEVES, AS",4,Intencionado,404,420,15,1,1509.0,430.0,36034,6550.0,4580.0,"(42.15769512, -8.39708026)",2015-09,09,2015,FRIEIRA,MIÑO,44.0,466.00,42.211952,-8.199392,17.369637,17.369637,1.0,2015-09-01,PONTEVEDRA,MEDER,42.118540,-8.442646,17.8,25.8,12.0,90.0,68.0,88.0,40.0,2.0,999.0,10.0,0.0,5.750805,5.750805
357743,2015360770,5.94,2015-09-08 00:00:00-04:00,42.554722,-8.018586,36,PONTEVEDRA,16,DOZÓN,4,Intencionado,87,194,18,3,1541.0,0.0,36016,7423.0,4702.0,"(42.55472219, -8.01858582)",2015-09,09,2015,PORTODEMOUROS,ULLA,297.0,1204.98,42.850000,-8.150000,34.544874,34.544874,0.0,2015-09-01,PONTEVEDRA,SERRA DO FARO,42.579758,-7.933075,13.3,18.2,9.6,117.0,76.0,90.0,55.0,19.0,905.0,9.0,0.0,7.535815,7.535815
357778,2015360794,3.20,2015-09-28 00:00:00-04:00,42.243441,-8.322375,36,PONTEVEDRA,13,COVELO,4,Intencionado,89,428,21,4,5805.0,0.0,36013,13190.0,10256.0,"(42.24344138, -8

In [15]:
df_final.groupby(['id']).agg({'id':'count'})

,id
id,
2001150021,1
2001150088,1
2001150090,1
2001150094,1
2001150111,1
...,...
2015360747,1
2015360751,1
2015360770,1


In [16]:
df_final.to_csv('datos_rios_embalses_tiempo.csv')